# Inverse kinematics of a 6dof manipulator
Consider the following manipulator, but with a 3dof wrist joint attached to the 
end of link 2.

<img src="https://github.com/robotica-cem/cinematica-notebooks/blob/main/figures/3d-3dof-revolute.png?raw=true" width=500 />

The following shows an example of solving this using the KDL library

See [KDL ChainIKSolverPos documentation](http://docs.ros.org/en/indigo/api/orocos_kdl/html/classKDL_1_1ChainIkSolverPos.html).

In [ ]:
!git clone https://github.com/orocos/orocos_kinematics_dynamics.git
!apt install libeigen3-dev libcppunit-dev # Without sudo on colab
!mkdir orocos_kinematics_dynamics/orocos_kdl/build
%cd orocos_kinematics_dynamics/orocos_kdl/build
!cmake ..
!make
!make install
%cd ../../python_orocos_kdl
!git submodule update --init
!mkdir build
%cd build
!cmake ..
!make
!make install
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/lib
!ldconfig
!python3 ../tests/PyKDLtest.py

In [1]:
import numpy as np
import doctest
import PyKDL as kdl
#import sympy as sy
#sy.init_printing()

## The kinematic chain

In [37]:
class SixDof:
    '''
    Represents the robotic arm pictured in the image above. It is  a robot with two perpendicular
        revolute joints connecting the first link to the base, and one revolute joint
        connecting link one and two. In its default position (for all joint angles 
        equal to zero), the first link is aligned with the global z-axis (vertical) and the 
        second link is in the direction of the positive global x-axis.
    '''
    def __init__(self, l1=2, l2=1):
        arm = kdl.Chain()
        arm.addSegment(kdl.Segment(joint=kdl.Joint(type=kdl.Joint.RotZ)))
        arm.addSegment(kdl.Segment(joint=kdl.Joint(type=kdl.Joint.RotY),
                      f_tip=kdl.Frame(kdl.Rotation.Identity(), 
                                     kdl.Vector(0, 0, l1))))
        arm.addSegment(kdl.Segment(joint=kdl.Joint(type=kdl.Joint.RotY),
                      f_tip=kdl.Frame(kdl.Rotation.Identity(), 
                                     kdl.Vector(l2, 0, 0))))
        arm.addSegment(kdl.Segment(joint=kdl.Joint(type=kdl.Joint.RotX)))
        arm.addSegment(kdl.Segment(joint=kdl.Joint(type=kdl.Joint.RotY)))
        arm.addSegment(kdl.Segment(joint=kdl.Joint(type=kdl.Joint.RotZ)))
        
        self.arm = arm
        self.fksolver = kdl.ChainFkSolverPos_recursive(self.arm)
        self.iksolver = kdl.ChainIkSolverPos_LMA(self.arm)
        
    def fk(self, th):
        '''
        Implements the forward kinematics.

        Arguments
        ---------
        th : array-like (3 or 6)
           Angle in radians of the degrees of freedoms, respectively.
  
        Returns
        -------
        wp : tuple (3)
           The position in the global frame of the wrist point (end of link 2)
        T_BE : kdl.Frame
           The transformation between End-effector frame and Base frame.
        
        Tests
        ------
        >>> l1, l2 = 2, 1
        >>> arm = SixDof(l1=l1, l2=l2)
        >>> # 1) End-effector pose at default position
        >>> wp, T_BE = arm.fk([0, 0, 0])
        >>> x, y, z = wp
        >>> "(%0.2f, %0.2f, %0.2f)" %(x, np.abs(y), z)
        '(1.00, 0.00, 2.00)'
        >>> # 2) End-effector pose at 90 degrees in all three joints
        >>> wp, T_BE = arm.fk([np.pi/2, np.pi/2, np.pi/2])
        >>> x, y, z = wp
        >>> "(%0.2f, %0.2f, %0.2f)" %(np.abs(x), y, np.abs(z))
        '(0.00, 1.00, 0.00)'
        >>> # 3) End-effector pose at 0 degress in first joint and 90 degress in second
        >>> wp, T_BE = arm.fk([0, np.pi/2, 0])
        >>> x, y, z = wp
        >>> "(%0.2f, %0.2f, %0.2f)" %(x, np.abs(y), z)
        '(2.00, 0.00, -1.00)'
        
        '''
        # The position of the tool center point
        ee_frame = kdl.Frame()
        th_ = kdl.JntArray(6)
        th_[3] = 0
        th_[4] = 0
        th_[5] = 0
        for i_ in range(len(th)):
            th_[i_] = th[i_]
        
        res = self.fksolver.JntToCart(th_, ee_frame)
        if res < 0:
            raise RuntimeError("Forward kinematics solver failed")
        tcp0 = kdl.Vector(0,0,0)
        tcp = ee_frame * tcp0
        
        return (tcp.x(), tcp.y(), tcp.z()), ee_frame
    
    def ik(self, wp, th_init):
        '''
        Implements the inverse kinematics.

        Arguments
        ---------
        wp : array-like (3) 
           Coordinates of the wrist point (tip of second link) in the base frame.
        th_init : array-like (3)
           Initial guess
        Returns
        -------
        th : tuple (3)
           The three first joint angles.
           
        Tests
        ------
        >>> l1, l2 = 2, 1
        >>> arm = SixDof(l1=l1, l2=l2)
        >>> # 1) Arm stretched out along the x-axis
        >>> wp = [3,0,0]
        >>> thtrue = [0, np.pi/2, -np.pi/2]
        >>> thinit = [0, np.pi/4, -np.pi/4]
        >>> th = arm.ik(wp, thinit)
        >>> th, thtrue
        >>> np.allclose(th, thtrue, atol=1e-2)
        True
        >>> # 2) Arm just rotated 90 degrees about first joint
        >>> wp = [0,1,2]
        >>> thtrue =  [np.pi/2, 0, 0] 
        >>> thinit = [0, 0, 0]
        >>> th = arm.ik(wp, thinit)
        >>> th, thtrue
        >>> np.allclose(th, thtrue, atol=1e-2)
        True
        >>> #  3) Use the forward map for some random angles. 
        >>> # Make intial guess reasonably close
        >>> ths = np.random.randn(3)
        >>> th1 = ths[0]
        >>> th2 = np.clip(ths[1], 0, np.pi)
        >>> th3 = np.clip(ths[2], -np.pi/2, np.pi/2)
        >>> thtrue = [th1, th2, th3]
        >>> thinit = [0.5*th1, 0.5*th2, 0.5*th3]
        >>> wp, T_BE = arm.fk(thtrue)
        >>> th = arm.ik(wp, thinit)
        >>> th, thtrue
        >>> np.allclose( th, thtrue, atol=1e-2)
        True
        '''
        # The position of the tool center point
        wp_ = kdl.Vector(wp[0], wp[1], wp[2])      
        target_frame = kdl.Frame(kdl.Rotation.Identity(), wp_)
        
        thinit_ = kdl.JntArray(6)
        thinit_[3] = 0
        thinit_[4] = 0
        thinit_[5] = 0
        for i in range(len(th_init)):
            thinit_[i] = th_init[i]

        th = kdl.JntArray(6)
        res = self.iksolver.CartToJnt(thinit_, target_frame, th)
        if res < 0:
            raise RuntimeError("Forward kinematics solver failed")
        
        return (th[0], th[1], th[2])

In [38]:
doctest.run_docstring_examples(SixDof.fk, globals())

In [39]:
doctest.run_docstring_examples(SixDof.ik, globals())

**********************************************************************
File "__main__", line 104, in NoName
Failed example:
    th, thtrue
Expected nothing
Got:
    ((0.0, 1.5694977714875114, -1.566900659213699), [0, 1.5707963267948966, -1.5707963267948966])
**********************************************************************
File "__main__", line 112, in NoName
Failed example:
    th, thtrue
Expected nothing
Got:
    ((1.5707963129520164, -6.999636753520856e-14, 1.0346248681377842e-13), [1.5707963267948966, 0, 0])
**********************************************************************
File "__main__", line 125, in NoName
Failed example:
    th, thtrue
Expected nothing
Got:
    ((-1.571845269680481, 1.0994759502508238, 1.3002332286371945), [-1.5718452696373943, 1.0994759541918722, 1.3002332328758082])
